In [27]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor
train_data = datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = ToTensor(), 
    download = False,            
)
test_data = datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = ToTensor()
)

In [2]:
print(train_data)


Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()


In [4]:
from torch.utils.data import DataLoader
loaders = {
    'train' : DataLoader(train_data, 
                         batch_size=100, 
                         shuffle=True, 
                         num_workers=1),
    
    'test'  : DataLoader(test_data, 
                         batch_size=100, 
                         shuffle=True, 
                         num_workers=1),
}
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7fe8fc961430>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7fe8fc96f730>}

In [13]:
import torch.nn as nn
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        # fully connected layer, output 10 classes
        self.out = nn.Linear(32 * 7 * 7, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output
    
    def extract_first_conv_features(self, x):
        x = self.conv1(x)
        return x

In [14]:
cnn = CNN()


In [15]:
loss_func = nn.CrossEntropyLoss()   


In [16]:
from torch import optim
optimizer = optim.Adam(cnn.parameters(), lr = 0.01)   

In [21]:
from torch.autograd import Variable

def train(num_epochs, cnn, loaders):
    
    cnn.train()
    total_step = len(loaders['train'])
        
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            
            # gives batch data, normalize x when iterate train_loader
            b_x = Variable(images)   # batch x
            b_y = Variable(labels)   # batch y

            output = cnn(b_x)             
            loss = loss_func(output, b_y)
            
            # clear gradients for this training step   
            optimizer.zero_grad()           
            
            # backpropagation, compute gradients 
            loss.backward()    
            # apply gradients             
            optimizer.step()                
            
            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

In [22]:
num_epochs = 10
train(num_epochs, cnn, loaders)


Epoch [1/10], Step [100/600], Loss: 0.1729
Epoch [1/10], Step [200/600], Loss: 0.1250
Epoch [1/10], Step [300/600], Loss: 0.0422
Epoch [1/10], Step [400/600], Loss: 0.1042
Epoch [1/10], Step [500/600], Loss: 0.0682
Epoch [1/10], Step [600/600], Loss: 0.1200
Epoch [2/10], Step [100/600], Loss: 0.0192
Epoch [2/10], Step [200/600], Loss: 0.0608
Epoch [2/10], Step [300/600], Loss: 0.1017
Epoch [2/10], Step [400/600], Loss: 0.1011
Epoch [2/10], Step [500/600], Loss: 0.0186
Epoch [2/10], Step [600/600], Loss: 0.1486
Epoch [3/10], Step [100/600], Loss: 0.0862
Epoch [3/10], Step [200/600], Loss: 0.0500
Epoch [3/10], Step [300/600], Loss: 0.0436
Epoch [3/10], Step [400/600], Loss: 0.0307
Epoch [3/10], Step [500/600], Loss: 0.0636
Epoch [3/10], Step [600/600], Loss: 0.0058
Epoch [4/10], Step [100/600], Loss: 0.0187
Epoch [4/10], Step [200/600], Loss: 0.0729
Epoch [4/10], Step [300/600], Loss: 0.0570
Epoch [4/10], Step [400/600], Loss: 0.0713
Epoch [4/10], Step [500/600], Loss: 0.1252
Epoch [4/10

In [85]:
def test():
    # Test the model
    cnn.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in loaders['test']:
            test_output = cnn(images)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            accuracy = (pred_y == labels).sum().item() / float(labels.size(0))
            pass
        
        print('Test Accuracy of the model on the 10000 test images: %.2f' % accuracy)
    pass
test()

Test Accuracy of the model on the 10000 test images: 0.98
